# Create Insert and Update script from Excel

## <span style="color: rgb(0, 128, 0);">1.&nbsp;Import&nbsp;packages</span>

In [ ]:
# 1. Import packages

import pandas as  pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import os

print("Packages imported")

## 2\. Create dataframe from raw data file

In [ ]:
# 2. Open raw data file
#---------------------------------------------------------------------------------------------------
    # REQUIRED - 
        # 1st row column name
        # 2nd row should contain datatype
        # Raw file should be present in the 1st sheet of the file

    # NOTES: Single quote (Global's) within the text will be replaced with additional quote (Global''s).
#---------------------------------------------------------------------------------------------------

# Open excel data file
data_file_loc = input("Enter data file location").lstrip().rstrip()
##data_file_loc = r'C:\Users\Koppakan\OneDrive - FM Global\Documents\ADO Cards\Sprint 15\Synthetic data for Service Order.xlsx'
print(data_file_loc)

data_file = pd.read_excel(data_file_loc)
data_file_df = pd.DataFrame(data_file)
# Remove trailing and ending spaces
data_file_df.replace(r"^ +| +$", r"", regex=True, inplace=True)
#.replace("'", "''", regex=True, inplace=True)

# Add another single quote to single quote within text
#data_file_df = data_file_df.replace("'", "''", regex=True, inplace=True)

data_file_df_1 = data_file_df
data_file_df_copy = data_file_df_1
#data_file_df.dtypes
print("File loaded")



## 3\. Add quotes to text datatype and remove quotes from NULL

In [ ]:
# 3. Data Preparation
# NOTES: Single quote (FM Global's) within the text will be replaced with additional quote (FM Global''s).

# Table name
##table_name = "main.service_order"
table_name = input("Enter table name ::")
print("table name :: ",table_name)

# Update on column name
##col_name = "service_order_id"
col_name = input("Enter primary key column name ::")
print("column name :: ",col_name)
col_name_df = data_file_df[col_name]

col_name_df1 = col_name+ " = " + data_file_df[col_name].astype(str)
col_name_df1

# Drop column from main dataframe
data_file_df = data_file_df.drop(col_name,axis =1)

# Create a list of column names
col_list = data_file_df.columns.tolist()
syn_data_update = pd.DataFrame()
syn_data_update1 = pd.DataFrame()

# Replace text datatypes to char - character varying, date, timestamp
data_file_df.iloc[:1] = data_file_df.iloc[:1].replace("timestamp", 'char', regex=True).replace("character varying", 'char', regex=True).replace("date", 'char', regex=True).replace("character", 'char', regex=True)

for i in range(len(col_list)):
    # Find value at location   
    dt_val = data_file_df.iat[0,i] 
  
# Add quotes to char datatype
    if  dt_val == "char":
        data_file_df[col_list[i]] = data_file_df[col_list[i]].str.replace("'", "''")
        new_col_nm = col_list[i] + "_new"                
        data_file_df = data_file_df.replace(np.nan, 'NULL', regex=True)
        data_file_df[new_col_nm] = " '" + data_file_df.iloc[:, i:i+1].astype(str)+"'"
        data_file_df[col_list[i]] = data_file_df[new_col_nm]
        data_file_df = data_file_df.drop(new_col_nm,axis =1)
        
# Replace 'nan'to blank and 'NULL' to NULL
data_file_df.replace({" 'NULL'": "NULL"}, inplace=True)

# Print result to excel
   print("Data preparation completed")




## 4\. Insert into script

In [ ]:
# insert into table
col_list_join = ','.join(col_list)
ins = "INSERT INTO " + table_name + "(" + col_list_join +")" + " VALUES("

# Combine all values in columns sep by ,
syn_data_insert = pd.DataFrame()
syn_data_insert['combined_cols'] = data_file_df[col_list].astype(str).agg(','.join, axis=1)

# Create insert statements
syn_data_insert['insert'] = ins + syn_data_insert['combined_cols'].astype(str) + ");"

# Print result to excel
export_file_name = "Insert_synthetic data for " + table_name +".xlsx"
syn_data_insert['insert'][1:].to_excel(export_file_name, sheet_name = "insert", index = False, header=False,)

file_loc = os.getcwd()+ "\\"+export_file_name
print("Insert script completed and copied to excel. \nLocation & File name:: ",file_loc)

## 5\. Update script

In [ ]:
#UPDATE table_name SET
# NOTES: Single quote (Global's) within the text will be replaced with additional quote (Global''s).

updt = "UPDATE " + table_name + " SET "

# Concatnate all columns & add ,
syn_data_update1 = pd.DataFrame()
for i in range(len(col_list)):
    syn_data_update = col_list[i] + " = " + data_file_df[col_list[i]].astype(str)    
    syn_data_update_df = pd.DataFrame(syn_data_update)
    syn_data_update1 = pd.concat([syn_data_update1,syn_data_update_df], axis = 1)

syn_data_update1['concat'] = pd.Series(syn_data_update1[col_list].values.tolist()).str.join(', ')

# WHERE column_name & add ;
syn_data_update1['update'] = updt + syn_data_update1['concat'].astype(str) + " WHERE " + col_name_df1.astype(str) + ";"

# Print result to excel
export_file_name1 = "Synthetic data for " + table_name +"_update"+".xlsx"
sheet = "update_" + table_name
syn_data_update1['update'][1:].to_excel(export_file_name1, sheet_name = sheet, header=False, index = False)
file_loc = os.getcwd()+ "\\"+export_file_name1
print("update script completed and copied to excel. \nLocation & File name:: ",file_loc)
